In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

2.4.0+cu124
True
0
NVIDIA GeForce GTX 1650


In [4]:
import os
import yaml

def show_yaml_file(yaml_file):
    if os.path.exists(yaml_file):
        try:
            with open(yaml_file, 'r') as file:
                data = yaml.safe_load(file)
            print("YAML file content:\n")
            print(yaml.dump(data, default_flow_style=False))
        except Exception as e:
            print(f"Error reading YAML file: {e}")
    else:
        print("File doesn't exist.")

# Specify the path to your yaml file
yaml_file_path = r'C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\data.yaml'
show_yaml_file(yaml_file_path)


YAML file content:

names:
- '0'
- '1'
nc: 2
test: ../test/images
train: ../train/images
val: ../valid/images



In [13]:
import os
import torch
from torchvision import transforms
from ultralytics import YOLO

# Load the model
model = YOLO('yolov8s.pt')
device = 'cuda:0'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define checkpoint save function
def save_checkpoint(model, epoch, path='C:\\Users\\priya\\OneDrive\\Desktop\\yolo_vd\\custom_dataset\\checkpoints'):
    os.makedirs(path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(path, f'checkpoint_epoch_{epoch}.pth'))

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0

    def check(self, current_loss):
        if self.best_loss is None:
            self.best_loss = current_loss
        elif current_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.best_loss = current_loss
            self.counter = 0
        return False

# Custom training function with checkpoint saving and early stopping
def train_with_early_stopping(model, data, epochs, batch_size, lr, device, checkpoint_interval=10, patience=10):
    model.to(device)
    
    early_stopping = EarlyStopping(patience=patience)

    # Configure and start training
    model.train(
        data=data,
        epochs=epochs,
        batch=batch_size,
        imgsz=640,
        device=device,
        optimizer='Adam',  # Explicitly set the optimizer
        lr0=lr,
        lrf=0.01,
        fliplr=0.0,
        amp=False
    )

    # Training loop for checkpoints and early stopping
    for epoch in range(epochs):
        if (epoch + 1) % checkpoint_interval == 0:
            save_checkpoint(model, epoch + 1)

        # Perform validation after each epoch
        results = model.val()
        current_loss = results['metrics/loss']

        # Check for early stopping
        if early_stopping.check(current_loss):
            print(f"Early stopping at epoch {epoch + 1}")
            break

# Define paths and parameters
data_path = r'C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\data.yaml'
epochs = 50
batch_size = 2
lr = 0.0001

# Run training with early stopping and checkpoint saving every 10 epochs
train_with_early_stopping(
    model,
    data=data_path,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr,
    device=device,
    checkpoint_interval=10  # Set to save checkpoints every 10 epochs
)


New https://pypi.org/project/ultralytics/8.2.76 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.74  Python-3.12.4 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\data.yaml, epochs=50, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None,

train: Scanning C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\train\labels.cache... 2951 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2951/2951 [00:00<?, ?it/s]
val: Scanning C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\valid\labels.cache... 142 images, 0 backgrounds, 0 corrupt: 100%|██████████| 142/142 [00:00<?, ?it/s]


Plotting labels to runs\detect\train7\labels.jpg... 
optimizer: Adam(lr=0.0001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      1.97G      1.641      2.174       1.81          4        640: 100%|██████████| 1476/1476 [05:42<00:00,  4.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:11<00:00,  3.21it/s]

                   all        142        246      0.597      0.353      0.446      0.246


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   3%|▎         | 1/36 [00:00<00:11,  3.01it/s]c:\Users\priya\OneDrive\Desktop\yolo_vd\myenv\Lib\site-packages\ultralytics\utils\plotting.py:1022: RuntimeWarning: invalid value encountered in cast
  for j, box in enumerate(boxes.astype(np.int64).tolist()):
Exception in thread Thread-97 (plot_images):
OverflowError: Python int too large to convert to C long

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\priya\OneDrive\Desktop\yolo_vd\myenv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\priya\OneDrive\Desktop\yolo_vd\myenv\Lib\site-packages\ultraly

                   all        142        246      0.686      0.556      0.649      0.416
                     0         69        122      0.526      0.393      0.438      0.296
                     1        121        124      0.846      0.718       0.86      0.536
Speed: 0.6ms preprocess, 67.3ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs\detect\train7
Ultralytics YOLOv8.2.74  Python-3.12.4 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\valid\labels.cache... 142 images, 0 backgrounds, 0 corrupt: 100%|██████████| 142/142 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 71/71 [00:03<00:00, 18.92it/s]


                   all        142        246      0.861      0.816      0.894       0.58
                     0         69        122      0.823      0.762      0.845      0.569
                     1        121        124        0.9      0.871      0.943      0.591
Speed: 0.8ms preprocess, 19.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\detect\train72


TypeError: 'DetMetrics' object is not subscriptable

In [11]:
import os
import cv2
from ultralytics import YOLO

# Paths provided by you
model_weights = r"C:\Users\priya\OneDrive\Desktop\yolo_vd\runs\detect\train33\weights\best.pt"
source_path = r"C:\Users\priya\OneDrive\Desktop\yolo_vd\INPUT_VID.mp4"
output_path = r"C:\Users\priya\OneDrive\Desktop\yolo_vd\OUTPUT_VID.mp4"

# Function to run inference on a video and save the output
def run_inference(model_weights, source, output):
    # Load the model
    model = YOLO(model_weights)
    
    # Open the input video
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output, fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run YOLOv8 inference on the frame
        results = model(frame)
        
        violence_detected = False
        for result in results:
            for box in result.boxes:
                confidence = box.conf.item()  # Get confidence score
                
                if box.cls == 1 and confidence > 0.60:  # Only consider violence with confidence > 65%
                    violence_detected = True
                    
                    # Draw bounding box
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    
                    # Overlay the label "Violence detected" with accuracy
                    label = f"Violence detected ({confidence * 100:.2f}%)"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
        if not violence_detected:
            # Overlay the label "Non-violence detected" if no violence is detected
            cv2.putText(frame, "Non-violence detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Write the frame to the output video
        out.write(frame)
    
    # Release everything when job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"Inference completed. Output saved to {output}")

if __name__ == "__main__":
    run_inference(model_weights, source_path, output_path)



0: 384x640 1 0, 16.0ms
Speed: 3.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 0, 13.1ms
Speed: 2.0ms preprocess, 13.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 14.8ms
Speed: 2.3ms preprocess, 14.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 19.4ms
Speed: 3.0ms preprocess, 19.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 12.8ms
Speed: 3.0ms preprocess, 12.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 13.7ms
Speed: 1.3ms preprocess, 13.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 12.8ms
Speed: 3.0ms preprocess, 12.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 13.1ms
S